In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
C:\Users\PC\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [10]:
params = utils.yaml_to_dict('config.yml')

## Primer paso: preprocesar los datos 
Hacerlo solo una vez

In [5]:
preprocessing.download_data(params)
preprocessing.extract_data(params)
preprocessing.generate_spectogram_images(params)
preprocessing.make_id_label_map(params)
preprocessing.split_data(params)

Window size is bigger than record segment: specgram_matrix_S-acelera11_segment24.jpg
Window size is bigger than record segment: specgram_matrix_S-acelera4_segment0.jpg
Window size is bigger than record segment: specgram_matrix_S-acelera5_segment11.jpg
Window size is bigger than record segment: specgram_matrix_S-acelera6_segment15.jpg
Window size is bigger than record segment: specgram_matrix_S-acelera7_segment16.jpg
Window size is bigger than record segment: specgram_matrix_S-acelera8_segment0.jpg
Window size is bigger than record segment: specgram_matrix_S-acelera8_segment1.jpg
Window size is bigger than record segment: specgram_matrix_S-arranca8_segment0.jpg
Window size is bigger than record segment: specgram_matrix_S-avanza9_segment22.jpg
Window size is bigger than record segment: specgram_matrix_S-frena10_segment24.jpg
Window size is bigger than record segment: specgram_matrix_S-frena5_segment28.jpg
Window size is bigger than record segment: specgram_matrix_S-izquierda7_segment6.jp

## Segund Paso: instanciar el modelo

In [6]:
inputs = tf.keras.layers.Input(shape=(224,224, 3))
net = model.ModelArchitecture(num_classes=params['num_classes'])
x = net(inputs, training=False)
net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 64)      9472      
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 64)      256       
_________________________________________________________________
activation (Activation)      (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 192)       110784    
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 192)       768       
_________________________________________________________________
activation_1 (Activation)    (None, 56, 56, 192)       0         
__________

## Tercer paso: crear el generador

In [12]:
generator = data.make_datagenerator(params, mode='training')
generator.n

Found 2416 images.


2416

## Cuarto paso: correr el modelo

In [11]:
optimizer = tf.keras.optimizers.Adam()

steps_per_epoch = generator.n // params['batch_size']
net.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
net.fit_generator(generator, steps_per_epoch=steps_per_epoch, epochs=params['num_epochs'])

Epoch 1/45
241/241 [==============================] - 8s 31ms/step - loss: 2.2987 - acc: 0.1162
Epoch 2/45
241/241 [==============================] - 7s 28ms/step - loss: 2.2990 - acc: 0.1154
Epoch 3/45
241/241 [==============================] - 7s 28ms/step - loss: 2.2990 - acc: 0.1112
Epoch 4/45
241/241 [==============================] - 7s 28ms/step - loss: 2.2981 - acc: 0.1220
Epoch 5/45
241/241 [==============================] - 7s 28ms/step - loss: 2.2995 - acc: 0.1187
Epoch 6/45
241/241 [==============================] - 7s 28ms/step - loss: 2.2978 - acc: 0.1154
Epoch 7/45
241/241 [==============================] - 7s 28ms/step - loss: 2.2977 - acc: 0.1162
Epoch 8/45
241/241 [==============================] - 7s 29ms/step - loss: 2.3004 - acc: 0.1120
Epoch 9/45
241/241 [==============================] - 7s 29ms/step - loss: 2.2989 - acc: 0.1162
Epoch 10/45
241/241 [==============================] - 7s 29ms/step - loss: 2.2980 - acc: 0.1170
Epoch 11/45
241/241 [==================